In [1]:
import pandas as pd
import numpy as np

In [2]:
from sqlalchemy import create_engine
import pymysql
from urllib.parse import quote
import tkinter, tkinter.ttk, pymysql, paramiko
from sshtunnel import SSHTunnelForwarder
from sqlalchemy import create_engine

from datetime import datetime, date, timedelta

pkeyfilepath = '/Users/apple/Desktop/ssh_key.pem' ######################### ssh_key.pem 파일 경로 넣으시면 됩니다.
mypkey = paramiko.RSAKey.from_private_key_file(pkeyfilepath)
sql_hostname = 'db-8j8d2-fkr.cdb.fin-ntruss.com'
sql_username = 'wkdtjdgus2142'            ######################### sql_username 입력하시면 됩니다.
sql_password = '?!0hLtR$_i8onochu=aC'            ######################### sql_password 입력하시면 됩니다.
sql_main_database = 'betterday_db'
sql_port = 3306
ssh_host = '10.0.1.14'
ssh_user = 'wkdtjdgus2142'                ######################### ssh_user 입력하시면 됩니다.
ssh_port = 22

# soslSuregi97o#rop0n0

In [3]:
# 전체고객
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
        sql = """
            SELECT ID FROM BDAY_CUST;
            """
        cur = con.cursor()
        cur.execute(sql)
        전체 = cur.fetchall()
            
전체 = pd.DataFrame(전체)
전체.columns = ['ID']

In [8]:
# 금투연금
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
        sql = """
            SELECT mut.BDAY_CUST_ID FROM MD_INVEST_ACCOUNTS mia, MD_USER_TOKEN mut
            WHERE mia.TOKEN_ID = mut.ID
            AND mia.ACCOUNT_TYPE = 105
            GROUP BY mut.BDAY_CUST_ID;
            """
        cur = con.cursor()
        cur.execute(sql)
        금투연금 = cur.fetchall()
            
금투연금 = pd.DataFrame(금투연금)
금투연금.columns = ['BDAY_CUST_ID']

In [7]:
#은행연금 및 펀드
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
        sql = """
            SELECT mut.BDAY_CUST_ID FROM MD_BANK_ACCOUNTS mba , MD_USER_TOKEN mut 
            WHERE ACCOUNT_TYPE IN (2001,2002,2003,2004,2999)
            AND (PROD_NAME LIKE '%펀드%')
            AND mba.TOKEN_ID = mut.ID
            GROUP BY mut.BDAY_CUST_ID; 
            """
        cur = con.cursor()
        cur.execute(sql)
        은행연금 = cur.fetchall()
            
은행연금 = pd.DataFrame(은행연금)
은행연금.columns = ['BDAY_CUST_ID']

In [5]:
# 연금보험
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
        sql = """
            SELECT mut.BDAY_CUST_ID FROM MD_INSU_INSURANCES mii , MD_USER_TOKEN mut 
            WHERE mii.INSU_TYPE = 10
            AND mii.INSU_STATUS = 02
            AND mii.IS_CONSENT =1
            AND mii.TOKEN_ID = mut.ID
            GROUP BY mut.BDAY_CUST_ID; 
            """
        cur = con.cursor()
        cur.execute(sql)
        연금보험 = cur.fetchall()
            
연금보험 = pd.DataFrame(연금보험)
연금보험.columns = ['BDAY_CUST_ID']

In [5]:
# 연금저축보험
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
        sql = """
            SELECT mut.BDAY_CUST_ID FROM MD_INSU_INSURANCES mii , MD_USER_TOKEN mut 
            WHERE mii.INSU_TYPE = 09
            AND mii.INSU_STATUS = 02
            AND mii.IS_CONSENT =1
            AND mii.TOKEN_ID = mut.ID
            GROUP BY mut.BDAY_CUST_ID;
            """
        cur = con.cursor()
        cur.execute(sql)
        연금저축보험 = cur.fetchall()
            
# 연금저축보험 = pd.DataFrame(연금저축보험)
# 연금저축보험.columns = ['BDAY_CUST_ID']

In [10]:
list(pd.DataFrame(연금저축보험)[0])

[10,
 132,
 157,
 454,
 513,
 648,
 727,
 779,
 895,
 977,
 1015,
 1059,
 1084,
 1086,
 1090,
 1095,
 1098,
 1154,
 1243,
 1248,
 1388,
 1480,
 1506,
 1589,
 1809,
 1891,
 1960,
 1967,
 2003,
 2033,
 2064,
 2129,
 2246,
 3196,
 5709,
 6951,
 7098,
 7303,
 7897,
 7910,
 7957,
 8251,
 8274,
 8294,
 8313,
 8336,
 8457,
 8536]

In [9]:
# CLUSTER2 - 연금납입금액 부족자 (납입금액 400만원 이하)
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
        sql = """
            SELECT mut.BDAY_CUST_ID, SUM(miap.PAID_IN_AMT) FROM MD_INVEST_ACCOUNTS_PENSION miap , MD_INVEST_ACCOUNTS mia , MD_USER_TOKEN mut 
            WHERE miap.INVEST_ACCOUNTS_ID =  mia.ID 
            AND mia.TOKEN_ID = mut.ID 
            AND mia.IS_CONSENT=1
            GROUP BY mut.BDAY_CUST_ID
            HAVING SUM(miap.PAID_IN_AMT) <= 4000000;
            """
        cur = con.cursor()
        cur.execute(sql)
        연금납입금액부족자 = cur.fetchall()
            
연금납입금액부족자 = pd.DataFrame(연금납입금액부족자)
연금납입금액부족자.columns = ['BDAY_CUST_ID','SUM_AMT']

In [10]:
# CLUSTER4 - 개인연금펀드 손실 발생자 
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)
        sql = """
            SELECT mut.BDAY_CUST_ID , COUNT(*), SUM(miap.EVAL_AMT - miap.PURCHASE_AMT) FROM MD_INVEST_ACCOUNTS_PRODUCTS miap , MD_INVEST_ACCOUNTS mia , MD_USER_TOKEN mut 
            WHERE miap.INVEST_ACCOUNTS_ID =  mia.ID 
            AND mia.TOKEN_ID = mut.ID 
            AND mia.IS_CONSENT=1
            AND miap.PROD_TYPE IN (413, 414, 415)
            AND miap.PURCHASE_AMT - miap.EVAL_AMT>0
            GROUP BY mut.BDAY_CUST_ID;
            """
        cur = con.cursor()
        cur.execute(sql)
        펀드손실 = cur.fetchall()
            
펀드손실 = pd.DataFrame(펀드손실)
펀드손실.columns = ['BDAY_CUST_ID','COUNT', 'LOST_AMT']

In [13]:
펀드손실

,BDAY_CUST_ID,COUNT,LOST_AMT
0,136,2,-98997.0
1,157,1,-4284.0
2,268,2,-1968.0
3,328,1,-4.0
4,438,3,-4184.0
...,...,...,...
174,8536,22,-305427.0
175,8537,1,-34.0
176,8553,1,-586.0
177,8558,1,-628.0


In [11]:
use_ins = list(금투연금.BDAY_CUST_ID)+list(은행연금.BDAY_CUST_ID) +list(연금저축보험.BDAY_CUST_ID) +list(연금보험.BDAY_CUST_ID) 
use_ins = list(set(use_ins))

전체['개인연금미가입자'] = 전체.ID.apply(lambda x: 'o' if x not in use_ins else np.nan)
전체['연금납입금액부족자'] = 전체.ID.apply(lambda x: 'o' if x in list(연금납입금액부족자.BDAY_CUST_ID) else np.nan)
전체['개인연금보험가입자'] = 전체.ID.apply(lambda x: 'o' if x in list(연금저축보험.BDAY_CUST_ID) else np.nan)
전체['개인연금펀드손실발생자'] = 전체.ID.apply(lambda x: 'o' if x in list(펀드손실.BDAY_CUST_ID) else np.nan)

In [12]:
전체 = 전체.sort_values(by='ID').reset_index().drop('index',axis=1)
전체.columns = ['전체고객', '개인연금미가입자', '연금납입금액부족자', '개인연금보험가입자', '개인연금펀드손실발생자']
전체.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6174 entries, 0 to 6173
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   전체고객         6174 non-null   int64 
 1   개인연금미가입자     5752 non-null   object
 2   연금납입금액부족자    151 non-null    object
 3   개인연금보험가입자    48 non-null     object
 4   개인연금펀드손실발생자  179 non-null    object
dtypes: int64(1), object(4)
memory usage: 241.3+ KB


In [64]:
# 전체.to_csv('연금_고객군집화.csv', encoding="utf-8-sig")

In [92]:
# cluster2 연금납입금액부족자 - 연금납입금액 전체 통계
연금납입금액부족자.SUM_AMT.describe()

count    9.200000e+01
mean     5.201679e+05
std      1.072869e+06
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.000000e+05
max      4.000000e+06
Name: SUM_AMT, dtype: float64

In [93]:
# cluster4 개인연금펀드손실발생자 - 손실 발생 건수 전체 통계
펀드손실.COUNT.describe()

count    112.000000
mean       3.116071
std        4.344916
min        1.000000
25%        1.000000
50%        1.000000
75%        3.000000
max       25.000000
Name: COUNT, dtype: float64

In [94]:
# cluster4 개인연금펀드손실발생자 - 손실금액 전체 통계
펀드손실.LOST_AMT.describe()

count    1.120000e+02
mean    -4.197017e+05
std      9.639101e+05
min     -6.768432e+06
25%     -2.292058e+05
50%     -4.780000e+03
75%     -2.135000e+02
max     -1.000000e+00
Name: LOST_AMT, dtype: float64

In [95]:
with SSHTunnelForwarder(
        (ssh_host, ssh_port),
        ssh_username=ssh_user,
        ssh_pkey=mypkey,
        remote_bind_address=(sql_hostname, sql_port)) as tunnel:
        con = pymysql.connect(host='127.0.0.1', user=sql_username,
            passwd=sql_password, db=sql_main_database,
            port=tunnel.local_bind_port)

        sql = """
            SELECT A.ID
            , CSS_BANK_9(A.ID)
            , CSS_BANK_15(A.ID)
            , CSS_CARD_1(A.ID)
            , CSS_INS_1(A.ID)
            FROM betterday_db.BDAY_CUST A
            GROUP BY A.ID
            ORDER BY A.ID ASC;
            """
        cur = con.cursor()
        cur.execute(sql)
        data = cur.fetchall()

In [96]:
data = pd.DataFrame(data)
data.columns = ['ID','예적금계좌보유','계좌액수','카드보유수','총보험']

In [97]:
data = pd.merge(data, 전체, how='left', left_on='ID', right_on = '전체고객').drop('전체고객',axis=1)

In [98]:
data

,ID,예적금계좌보유,계좌액수,카드보유수,총보험,개인연금미가입자,연금납입금액부족자,개인연금보험가입자,개인연금펀드손실발생자
0,10,0,274529.0,7,3,NaN,NaN,o,NaN
1,13,0,61127642.0,7,0,NaN,o,NaN,NaN
2,20,0,992.0,3,0,o,NaN,NaN,NaN
3,21,0,5897040.0,3,0,o,NaN,NaN,NaN
4,30,0,NaN,0,0,o,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5801,7604,3,4214890.0,4,0,o,NaN,NaN,NaN
5802,7605,0,NaN,0,0,o,NaN,NaN,NaN
5803,7606,0,NaN,0,0,o,NaN,NaN,NaN
5804,7607,0,NaN,0,0,o,NaN,NaN,NaN


In [99]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5806 entries, 0 to 5805
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           5806 non-null   int64  
 1   예적금계좌보유      5806 non-null   int64  
 2   계좌액수         4097 non-null   float64
 3   카드보유수        5806 non-null   int64  
 4   총보험          5806 non-null   int64  
 5   개인연금미가입자     5522 non-null   object 
 6   연금납입금액부족자    92 non-null     object 
 7   개인연금보험가입자    38 non-null     object 
 8   개인연금펀드손실발생자  112 non-null    object 
dtypes: float64(1), int64(4), object(4)
memory usage: 453.6+ KB
